<a href="https://colab.research.google.com/github/karen-wang/writing-samples/blob/main/Using_Lamini_to_Write_Engaging_Bios_for_Rescue_Animals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lamini + Social Good:** Using Lamini to Write Engaging Bios for Rescue Animals
_Karen Wang_ \
_Take-Home Test_ \
_July 10, 2023_

<img src="https://drive.google.com/uc?id=12OMknjfUPaAQLPB3BBdX95sSZ1Jes6b8" width="40%" alt="Sleeping orange tabby cat.">

Lamini is a Python package that allows users to train and use large language models (LLMs) for their specific use cases. Lamini makes generative AI more accessible and affordable for everyone.

Every year, millions of unwanted and abandoned animals are sent to shelters. In order to appeal to potential adopters, shelter workers must undergo the difficult and time-consuming task of writing engaging and unique bios for each animal. Today, we'll show you how to leverage the power of Lamini to make this task faster, easier, and more effective.

In [1]:
%%capture
# @title Setup

from google.colab import auth
import requests
import os
import yaml

def authenticate_powerml():
  auth.authenticate_user()
  gcloud_token = !gcloud auth print-access-token
  powerml_token_response = requests.get('https://api.powerml.co/v1/auth/verify_gcloud_token?token=' + gcloud_token[0])
  return powerml_token_response.json()['token']

production_token = authenticate_powerml()

config = {
    "production": {
        "key": production_token,
        "url": "https://api.powerml.co"
    }
}

keys_dir_path = '/root/.powerml'
os.makedirs(keys_dir_path, exist_ok=True)

keys_file_path = keys_dir_path + '/configure_llama.yaml'
with open(keys_file_path, 'w') as f:
  yaml.dump(config, f, default_flow_style=False)

First, authenticate with Lamini and install the necessary libraries.

In [2]:
!pip install --upgrade llama-llm==0.0.15

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 21.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyyaml: filename=PyYAML-5.4.1-cp310-cp310-linux_x86_64.whl size=45658 sha256=d5444abddd9803e1a51382445b5f845dda1375d8318ea0a091bbcd4fa4fdc757
  Stored in directory: /root/.cache/pip/wheels/c7/0d/22/696ee92245ad710f506eee79bb05c740d8abccd3ecdb778683
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0


In [3]:
!pip install --upgrade scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 22.8 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1


Next, import and initialize the LLM. We want to give it a descriptive name; in our case we will use the LLM to generate bios for shelter animals, so we will name it `animal_bios`.



In [36]:
from llama import LLM

llm = LLM(name="animal_bios")

Next, define the input and output type for your model.

The LLM module is designed to be a general-purpose model that can work with any natural language input and output formats. One example would be a question and answer format, where you give a question to the LLM and it outputs an answer.

For our purpose, we want the input to be information about an animal, and the output to be an engaging and descriptive bio of the animal.

Lamini inputs and outputs are defined as Python classes that inherit from the library's `Type` class. Each class attribute provides `Context` for the model to know what you're trying to do with it.

We define an input `Animal` and output `Bio` type below.

In [37]:
from llama import Type, Context

# Input
class Animal(Type):
  species: str = Context("the species of the animal")
  breed: str = Context("the breed of the animal")
  sex: str = Context("the sex of the animal (male or female)")
  coat: str = Context("the animal's coat pattern and color")

# Output
class Bio(Type):
  name: str = Context("the name of the animal")
  description: str = Context("a description of the animal")

The `Animal` input includes information about the animal's species, breed, sex, and coat. From this, we want to generate an appropriate name and description for the animal's bio.

In [38]:
animal = Animal(
    species = "cat",
    breed = "mixed",
    sex = "male",
    coat = "orange tabby"
)

animal_bio = llm(input=animal, output_type=Bio)
print(f"Name: {animal_bio.name}")
print(f"Bio: {animal_bio.description}")

Name: Tiger
Bio: Tiger is a handsome male orange tabby cat with a mixed breed. He has a soft, fluffy coat and bright, inquisitive eyes.


Now let's try adding more complex types to the input. In addition to basic information about an animal, we can add information about the animal's personality, which can be important for potential adopters to know. We can define the personality type as a list of traits and add it to the `Animal` class.

In [40]:
# Input
class Personality(Type):
  traits: list = Context("list of personality traits")

class Animal(Type):
  species: str = Context("the species of the animal (dog or cat)")
  breed: str = Context("the breed of the animal")
  sex: str = Context("the sex of the animal (male or female)")
  coat: str = Context("the animal's coat pattern and color")
  personality: Personality = Context("the animal's personality")

Now let's add some personality traits to our example and re-run the LLM.

In [41]:
personality = Personality(
    traits = ["friendly", "likes chicken"]
)

animal = Animal(
    species = "cat",
    breed = "mixed",
    sex = "male",
    coat = "orange tabby",
    personality = personality
)

animal_bio = llm(input=animal, output_type=Bio)
print(f"Name: {animal_bio.name}")
print(f"Bio: {animal_bio.description}")

Name: Orangey
Bio: Orangey is a friendly and curious male mixed cat with an orange tabby coat. He loves to explore and is particularly fond of chicken.


Since we've now specified that the animal is friendly and likes chicken, the LLM produces a bio that includes these traits. We can also tell the LLM to improve on certain criteria; for example, to add a note to potential adopters in the bio description.

In [42]:
llm.improve(on="description", to="add a note to potential adopters")

animal_bio = llm(input=animal, output_type=Bio)
print(f"Name: {animal_bio.name}")
print(f"Bio: {animal_bio.description}")

Name: Orangey
Bio: Orangey is a friendly and curious male mixed cat with an orange tabby coat. He loves to explore and is particularly fond of chicken. He is an affectionate and loyal companion and would make a great addition to any family.


Since shelters deal with influxes of multiple animals at a time, we can use Lamini to generate multiple outputs at once. In our case, we can give it a list of cats and their corresponding information, and tell Lamini to do the work of generating bios for all of them.

In [44]:
cats = [
    Animal(
    species = "cat",
    breed = "mixed",
    sex = "male",
    coat = "orange tabby",
    personality = Personality(traits=["friendly", "likes chicken"])
  ),
    Animal(
    species = "cat",
    breed = "mixed",
    sex = "male",
    coat = "black",
    personality = Personality(traits=["easygoing", "likes fish"])
  ),
    Animal(
    species = "cat",
    breed = "mixed",
    sex = "female",
    coat = "orange tabby",
    personality = Personality(traits=["shy", "loves to cuddle"])
  )
]

for cat in cats:
  animal_bio = llm(input=cat, output_type=Bio, random=True)
  print(f"Name: {animal_bio.name}")
  print(f"Bio: {animal_bio.description}")

Name: Orangey
Bio: Orangey is a friendly and active male mixed cat with an orange tabby coat. He loves chicken and is always looking for a playmate. He is a great companion and would make a wonderful addition to any family.
Name: Blackie
Bio: Blackie is a male cat of a mixed breed with a black coat. He is an easygoing and fish-loving cat who would make a great companion for any family. Potential adopters should note that Blackie loves to cuddle and will often follow you around the house.
Name: Candy
Bio: Candy is a female orange tabby cat who is shy and loves to cuddle. She is a mixed breed with a beautiful coat and a sweet personality. She would make a great companion for someone looking for a loyal and loving pet.


Another way of improving the LLM output is to give it existing data to learn from. In this case, we've taken data from the Humane Society of Silicon Valley (HSSV) website and given it to the LLM to train on. This will produce bios that are more aligned with the tone and style of HSSV.

In [11]:
# Source: Humane Society of Silicon Valley
animal = Animal(
    species = "cat",
    breed = "mixed",
    sex = "female",
    coat = "siamese",
    personality = Personality(traits=["curious",
                                      "high energy",
                                      "likes snuggling"])
  )

bio = Bio(
    name = "Bluebell",
    description = """Hi there, Bluebell here. I'm a cute little kitty looking
    for a forever home. I am curious and love exploring my surroundings.
    Despite my high energy levels, I am still a lap kitty at heart and love
    nothing more than snuggling up with my humans, esp if you massage my head
    and neck. I am still learning my manners and have a tendency to give love
    nibbles, but I am a quick learner and am eager to please. So come me visit
    me soon."""
)

data = [animal, bio]
llm.add_data(data)

In conclusion, we've used Lamini to develop a scalable solution for generating bios for shelter animals. We've also taught it to iteratively improve on certain criteria, by using both specific prompts and existing data.

This is just one of the many ways that Lamini can be used for contributions to social good. Check out the [Lamini documentation](https://lamini-ai.github.io/) for more examples and ideas for future projects.